# Project 1: Preparing an image dataset for model training

**Goal:**

- Implement a [map-style Dataset class](https://pytorch.org/docs/stable/data.html#map-style-datasets).
- Implement a [data loader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) that directly feeds data into model training.
- Quality check your data, served through dataset class.

**Acceptance criteria:**

- The Dataset class should implement `__getitem__`  and `__len__` methods, where `__getitem__` should return a training example, which is a tuple of (image, label).
- The data loader should be iterable and returns batches of examples.
- In a notebook, present example images, labels and ensure correct shapes by visual inspection.

## Step 1: Create a Dataset class

One of the first things you may need to do in a practical project is to create a custom dataset class. As a start, let us use CIFAR10 raw data as an example to create one. First, we need to download the data:


In [5]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzf cifar-10-python.tar.gz

Inspect the downloaded raw data and folder structure:

In [12]:
!ls -lht cifar-10-batches-py/

total 363752
-rw-r--r--  1 zzsi  staff    88B Jun  4  2009 readme.html
-rw-r--r--  1 zzsi  staff   158B Mar 30  2009 batches.meta
-rw-r--r--  1 zzsi  staff    30M Mar 30  2009 data_batch_4
-rw-r--r--  1 zzsi  staff    30M Mar 30  2009 data_batch_1
-rw-r--r--  1 zzsi  staff    30M Mar 30  2009 data_batch_5
-rw-r--r--  1 zzsi  staff    30M Mar 30  2009 data_batch_2
-rw-r--r--  1 zzsi  staff    30M Mar 30  2009 data_batch_3
-rw-r--r--  1 zzsi  staff    30M Mar 30  2009 test_batch


The archive contains the files data_batch_1, data_batch_2, ..., data_batch_5, as well as test_batch. Each of these files is a Python "pickled" object.

Loaded using the `unpickle` method provided below, each of the batch files contains a dictionary with the following elements:

*data* -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

*labels* -- a list of 10000 numbers in the range 0-9. 

**Your task**: Fill in the missing code in the next cell, so that it can pass the test that follows.

In [20]:
import pickle


def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def reshape_numpy_image(one_numpy_image):
    # TODO: which of the following is correct?
    shape_with_channels_last = (32, 32, 3)
    shape_with_channels_first = (3, 32, 32)
    return one_numpy_image.reshape(*shape_with_channels_last)


class CustomDataset:
    def __getitem__(self, index):
        ...
    
    def __len__(self):
        ...

In [21]:
# This is the test code for the class CustomDataset.

def test_custom_dataset():
    ds = CustomDataset()
    print(f"Dataset has {len(ds)} examples")
    assert len(ds) > 0, f"Your dataset has no examples"
    first_example = ds[0]
    print(f"The first example is a {type(first_example)} with {len(first_example)} elements")
    image, label = first_example
    print(f"The first example has an image of shape {image.shape} and label {label}")
    assert len(image.shape) == 3
    assert (label >= 0) and (label < 10), f"Label {label} is not in [0, 10)"

test_custom_dataset()

If it works, congratulations! You have completed the first step of the project.


## Step 2: Visualize the image data

However, passing the unit tests may not be enough. The content of the image may not make sense even if the shape is correct, depending on how the reshaping of numpy arrays is done.

In the next cells, display several images from the dataset together with the label. Visually check the image and the label. A helper function is provided.

In [ ]:
from matplotlib import pyplot as plt

import numpy as np

def display_image_with_label(image: np.array, label: int):
    plt.imshow(image)
    plt.title(f"Label: {label}")
    plt.show()

# TODO: add code to display at least 2 images from CustomDataset.
# You can use one cell for each image.

In [ ]:
# TODO: display the first image and label

In [ ]:
# TODO: display the second image and label

#### Optional practice: visualizing images in html

You can also save images as files, and then use the [`display`](https://ipython.org/ipython-doc/3/interactive/html/widgets.html#widgets.display) function to display them in an IPython notebook.

Example code to save a numpy array as an image:

```{python}
from PIL import Image

Image.fromarray(image).save('image.png')
```


## Step 3: Create a data loader

A data loader is a class that can be used to load batches of data. It is a simple wrapper around a dataset that provides a way to iterate over the dataset, returning a batch of data at each iteration.

For PyTorch, we strongly recommend using the [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class to load the data. It has a lot of built-in functionalities to handle efficient loading of data that you don't need to implement from scratch. In our experience, it is rarely needed to implement a custom version of `DataLoader`. Getting familar with how to use a PyTorch `DataLoader` is usually enough.

**Your task**: Fill in the missing code in the next cell, so that it can pass the test that follows.

In [ ]:
from torch.utils.data import DataLoader

batch_size = 8

# TODO: fill in the code to create a data loader with a batch size of 8.
data_loader = DataLoader(...)

In [ ]:
def test_data_loader():
    for batch in data_loader:
        images, labels = batch
        assert images.shape[0] == batch_size, f"Expecting a batch of {batch_size} images"
        assert len(labels) == batch_size, f"Expecting a batch of {batch_size} labels"
        break


test_data_loader()

#### Optional practice: display images in a grid using `matplotlib`

You can also display multiple images in one figure. This way you can visualize a batch of images easily. Below is a function to start with.

```
def show_images(images, ncols=8):
    nrows = (len(images) + ncols - 1) // ncols
    plt.figure(figsize=(2 * ncols, 2 * nrows))
    for i in range(len(images)):
        plt.subplot(nrows, ncols, i + 1)
        plt.imshow(images[i])
        plt.axis('off')
```

## Step 4 (optional): Writing unit tests for your dataset class

To make sure your dataset is feeding the right data to the model, visualization is helpful, but unit tests can make certain checks more efficient.

You may have noticed, in previous steps unit tests such as `test_custom_dataset` are already used to check the correctness of your implementation.

The following types of unit tests can be written against a dataset class:

- Tests that check the functionality of the class (e.g. calling `len(dataset)` does not raise an error).
- Tests that check the type and shape of the data returned by the class.
- Tests that check the content of the data returned by the class.

Below is an example unit test that checks if the shape of the image is "channels-first" (3, height, width) rather than "channels-last" (height, width, 3). PyTorch expects the image to be in "channels-first" format, while Tensorflow expects it to be "channels-last".

**Your task**: Fill in the missing code in unit test. Make necesssary changes to your dataset class to pass the test. Can you think of other tests that can answer questions about the dataset? For example, can you write a test about the maximum pixel intensity of this dataset?



In [ ]:
def test_custom_dataset_returns_images_with_channels_first():
    """Check the shape of the image, especially the channels axis.
    """
    # TODO: fill in.
    pass


test_custom_dataset_returns_images_with_channels_first()

Congratulations! You have completed the first project.